In [9]:
import pandas as pd
import numpy as np
import gpxpy
import gpxpy.gpx
import folium


In [4]:
def process_gpx_to_df(file_name):
    gpx = gpxpy.parse(open(file_name)) 
    
    #(1)make DataFrame
    track = gpx.tracks[0]
    segment = track.segments[0]
    # Load the data into a Pandas dataframe (by way of a list)
    data = []
    segment_length = segment.length_3d()
    for point_idx, point in enumerate(segment.points):
        data.append([point.longitude, point.latitude,point.elevation,
        point.time, segment.get_speed(point_idx)])
        columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
        gpx_df = pd.DataFrame(data, columns=columns)
        
    #2(make points tuple for line)
    points = []
    for track in gpx.tracks:
        for segment in track.segments: 
            for point in segment.points:
                points.append(tuple([point.latitude, point.longitude]))
        
    return gpx_df, points

In [11]:
df, points = process_gpx_to_df('00c6622.gpx')


In [16]:
df


,Longitude,Latitude,Altitude,Time,Speed
0,24.538536,60.156789,None,2022-01-06 10:00:00+00:00,2.583685
1,24.538660,60.156691,None,2022-01-06 10:00:05+00:00,3.310236
2,24.539017,60.156565,None,2022-01-06 10:00:11+00:00,4.166932
3,24.539390,60.156427,None,2022-01-06 10:00:17+00:00,3.797710
4,24.539679,60.156322,None,2022-01-06 10:00:23+00:00,3.374952
...,...,...,...,...,...
404,24.643033,60.144210,None,2022-01-06 10:28:47+00:00,4.008721
405,24.643170,60.144185,None,2022-01-06 10:28:49+00:00,4.005369
406,24.643359,60.144135,None,2022-01-06 10:28:52+00:00,3.994623
407,24.643485,60.144098,None,2022-01-06 10:28:54+00:00,4.115606


In [12]:

mymap = folium.Map( location=[ df.Latitude.mean(), df.Longitude.mean() ], zoom_start=6, tiles=None)
folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)

In [13]:
folium.PolyLine(points, color='red', weight=4.5, opacity=.5).add_to(mymap)

In [15]:
html_camino_start = """
 Start of day {camino_day}
 """.format(camino_day=camino_day)
popup = folium.Popup(html_camino_start, max_width=400)
 #nice green circle
folium.vector_layers.CircleMarker(location=[lat_start, long_start], radius=9, color='white', weight=1, fill_color='green', fill_opacity=1, popup=html_camino_start).add_to(mymap) 
 #OVERLAY triangle
folium.RegularPolygonMarker(location=[lat_start, long_start], 
 fill_color='white', fill_opacity=1, color='white', number_of_sides=3, 
 radius=3, rotation=0, popup=html_camino_start).add_to(mymap)

NameError: name 'camino_day' is not defined

In [69]:

import streamlit as st
import gpxpy
import folium
from streamlit_folium import st_folium

def parse_gpx(file_path, color='red', number = 0):
    gpx_file = open(file_path, 'r')
    gpx = gpxpy.parse(gpx_file)
    points = []
    
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                points.append({'Latitude': point.latitude, 'Longitude': point.longitude, 'Color': color, 'Number': number})
        
    return points

def read_files():
    gpx_files = ["00c6622.gpx", '0a7b419.gpx', '0ab9d37.gpx']  # Add your GPX file paths here
    colors = {'00c6622.gpx': 'red', '0a7b419.gpx': 'blue', '0ab9d37.gpx': 'green'}  # Colors for each GPX file

    # Parse each GPX file
    all_points = []
    number = 0 
    for file, color in colors.items():
        points = parse_gpx(file, color, number)
        # for point in points:
        #     points.append({'Latitude': point.latitude, 'Longitude': point.longitude, 'Color': color})
        number += 1
        all_points.extend(points)
    return all_points

def plot_map(points):
    m = folium.Map(location=[points[0]['Latitude'], points[0]['Longitude']], zoom_start=12)
    seen = {}
    for point in points:
        if not seen.get(point['Number'], False):
            print(seen)
            folium.Marker(location=[point['Latitude'], point['Longitude']],
                      icon=folium.Icon(color=point['Color'], icon='info-sign')
                      ).add_to(m)
        seen[point['Number']] = True
    return m

all_points = read_files()


In [70]:
all_points

[{'Latitude': 60.156789226,
  'Longitude': 24.538536267,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.156690907,
  'Longitude': 24.538660152,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.156564927,
  'Longitude': 24.539016634,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.156426625,
  'Longitude': 24.539389964,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.156322354,
  'Longitude': 24.539679307,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.156207522,
  'Longitude': 24.539973428,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.156033346,
  'Longitude': 24.540373916,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.155974422,
  'Longitude': 24.540516157,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.155801838,
  'Longitude': 24.540863251,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.155726401,
  'Longitude': 24.540905161,
  'Color': 'red',
  'Number': 0},
 {'Latitude': 60.155458599,
  'Longitude': 24.541015467,
  'Color': 'red',
  'Number': 0},

In [71]:
plot_map(all_points)

{}
{0: True}
{0: True, 1: True}


In [59]:
seen[0] = 0
seen[3] = 9
seen.get(9, False)

False